In [1]:
import os
import pandas as pd
from autoemulate.core.compare import AutoEmulate
from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from ModularCirc import BatchRunner
import torch
from autoemulate.core.save import ModelSerialiser
from autoemulate.core.logging_config import get_configured_logger
import matplotlib.pyplot as plt

from comparative_gsa.utils import helpers
figsize = (9, 5)

In [2]:
param_filename = 'parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas'

n_samples = 245760

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

parameters_json_file = os.path.join(simulation_out_path, 'parameters.json')

# Load the simulation input parameters
input_params = pd.read_csv(os.path.join(simulation_out_path, f'input_samples_{n_samples}.csv'))

# Load the summary statistics
summary_stats = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

output_to_emulate = 'p_ao_max'

emulators_path = os.path.join(simulation_out_path, 'emulators', output_to_emulate)

In [3]:
# Use ModularCirc's BatchRunner to condense parameter names. This requires setting up the sampler first.
br = BatchRunner()

br.setup_sampler(parameters_json_file)

br._parameters_2_sample

FileNotFoundError: [Errno 2] No such file or directory: '../outputs/simulations/output_245760_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/parameters.json'

In [ ]:
# Get column names for parameters up to 'lv.k_pas'
parameter_names = list(input_params.columns[:input_params.columns.get_loc('lv.k_pas') + 1])

In [ ]:
# Extract parameter names and bounds from the dictionary
parameters_range = br._parameters_2_sample

parameter_names = list(parameters_range.keys())
parameter_bounds = list(parameters_range.values())

parameter_bounds

[(1950.0, 3250.0),
 (0.22499999999999998, 0.375),
 (843.75, 1406.25),
 (2.25, 3.75),
 (6.75, 11.25),
 (84.975, 141.625),
 (4.5, 7.5),
 (3.0749999999999997, 5.125),
 (0.22499999999999998, 0.375),
 (0.41250000000000003, 0.6875),
 (0.00666, 0.03),
 (0.75, 1.25),
 (2.25, 3.75),
 (0.00666, 0.03)]

In [ ]:
problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds,
}
ae_results = helpers.ae_load_result(os.path.join(emulators_path, 'GaussianProcess_0_20250821_142302'))

In [ ]:
predictions = ae_results.model.predict()